In [ ]:
import pandas as pd
import os
import re

import os
import shutil
import pandas as pd
import numpy as np

from data_processing.pose_features.utils import euclidean_distance
from data_processing.pose_features.pose_geometry import pose_distances


In [8]:
segment_content = pd.read_excel('/home/gerardo/LSE_HEALTH/LSE-Health-UVigo.xlsx', sheet_name='SegmentsContent')
glosses_content = pd.read_excel('/home/gerardo/LSE_HEALTH/LSE-Health-UVigo.xlsx', sheet_name='GlossesContent')

signer_video = pd.read_excel('/home/gerardo/LSE_HEALTH/LSE_TFG/video_processing/signer_video.xlsx')

labels = pd.read_csv('/home/gerardo/LSE_HEALTH/LSE_TFG/labeled_words.csv')

label2code = labels.set_index("Word")["Encoded_Label"].to_dict()

In [5]:
os.mkdir('/home/gerardo/LSE_SEGMENTATION_DATASET')

In [ ]:
def process_file(df):

    output_rows = []

    coord_cols = [c for c in df.columns if re.search(r'_(x|y|z)$', c)]

    landmarks = sorted({col.rsplit('_', 1)[0] for col in coord_cols})


    for _, row in df.iterrows():
        dict_coords = {'frame': row['frame']}


        for name in landmarks:
            try:
                x = row[f'{name}_x']
                y = row[f'{name}_y']
                z = row[f'{name}_z']
                dict_coords[name] = (x, y, z)
            except KeyError:

                continue

        try:
            d1 = euclidean_distance(dict_coords['pose.LEFT_HIP'], dict_coords['pose.LEFT_SHOULDER'])
            d2 = euclidean_distance(dict_coords['pose.RIGHT_HIP'], dict_coords['pose.RIGHT_SHOULDER'])
            height_percentage = ((d1 + d2) / 2) * 0.265

            distances = pose_distances(dict_coords)
            for k in distances:
                distances[k] = distances[k] / height_percentage

            gesture_value = row.get('Gesture', 0)
            frame_data = {'frame': row['frame'], 'Gesture': gesture_value, **distances}
            output_rows.append(frame_data)

        except KeyError as e:
            print(f"Missing keypoint in frame {row['frame']}: {e}")
            continue
    df_out = pd.DataFrame(output_rows)
    return df_out


In [ ]:
path_spot_dataset = '/home/gerardo/LSE_SPOT_DATASET'
fps = 25

for file in os.listdir(path_spot_dataset):
    if file.endswith('_4.csv'):
        continue
    df = pd.read_csv(os.path.join(path_spot_dataset, file))
    video_name = re.split('_[0-9]*.csv', file)[0]

    segments = segment_content[segment_content['File'] == video_name]
    segments = segments[['Start(ms)', 'End(ms)']]
    

    glosses = glosses_content[glosses_content['File'] == video_name]

    for _, segment in segments.iterrows():

        glosses_per_segment = glosses.loc[
            (glosses['Start(ms)'].between(segment['Start(ms)'], segment['End(ms)'])) &
            (glosses['End(ms)'].between(segment['Start(ms)'], segment['End(ms)']))
        ]

        glosses_per_segment = glosses_per_segment.copy()


        glosses_per_segment['Start(ms)'] = (glosses_per_segment['Start(ms)'] / 1000 * fps).astype(int)
        glosses_per_segment['End(ms)']   = (glosses_per_segment['End(ms)']   / 1000 * fps).astype(int)
        truth_frames = []
        for _, frames in glosses_per_segment.iterrows():
            for i in range(frames['Start(ms)'], frames['End(ms)']):
                try:
                    gloss = frames['Gloss'].replace('*', '')
                    label = label2code[gloss]
                    truth_frames.append(i)
                except KeyError:
                    continue
                

        start_frame = int(segment['Start(ms)'] / 1000 * fps)
        end_frame = int(segment['End(ms)'] / 1000 * fps)

        df_segment = df.loc[df['frame'].between(start_frame, end_frame)]

        gesture_list = []
        for _, frame in df_segment.iterrows():
            if frame['frame'] in truth_frames:
                try:
                    gloss = glosses_per_segment.loc[
                                (glosses_per_segment['Start(ms)'] <= frame['frame']) &
                                (glosses_per_segment['End(ms)']   >  frame['frame'])
                            ]['Gloss'].iloc[0].replace('*', '')
                    label = label2code[gloss]
                except (KeyError, IndexError):
                    label = 43
            else:
                label = 43

            gesture_list.append(label)
        
        if all(l == 43 for l in gesture_list):
            continue
        else:
            df_segment = df_segment.copy()
            df_segment['Gesture'] = gesture_list

            f_df = process_file(df_segment)

            f_df.to_csv(os.path.join('/home/gerardo/LSE_SEGMENTATION_DATASET',
                                    f'{video_name}_{int(segment["Start(ms)"])}_{int(segment["End(ms)"])}.csv'),
                                    index=False)

In [ ]:
def downsample_by_boxes(df: pd.DataFrame, target_length: int) -> pd.DataFrame:
    n = len(df)
    if n <= target_length:
        return df.reset_index(drop=True)
    box_size = n / target_length
    indices = [min(n-1, int(round((i + 0.5) * box_size)))
               for i in range(target_length)]
    return df.iloc[indices].reset_index(drop=True)

def process_file(input_path: str,
                 output_path: str,
                 target_gesture: int,
                 target_len_per_run: int,
                 gesture_col: str = 'Gesture'):
    """
    Read input_path, downsample only the runs of target_gesture,
    reassemble and write to output_path.
    """
    df = pd.read_csv(input_path)
    run_id = df[gesture_col].ne(df[gesture_col].shift()).cumsum()
    processed_runs = []
    for _, run in df.groupby(run_id):
        if run[gesture_col].iloc[0] == target_gesture:
            run_ds = downsample_by_boxes(run, target_len_per_run)
        else:
            run_ds = run.reset_index(drop=True)
        processed_runs.append(run_ds)

    df_out = pd.concat(processed_runs, ignore_index=True)
    df_out.to_csv(output_path, index=False)

def split_and_prepare(input_dir: str,
                      output_dir: str,
                      target_gesture: int,
                      target_len_per_run: int,
                      splits=(0.7, 0.15, 0.15),
                      seed=42):
    """
    Splits the CSVs in input_dir into train/val/test according to `splits`,
    downsampling only the train set, and copies val/test untouched.
    """
    files = [f for f in os.listdir(input_dir) if f.lower().endswith('.csv')]
    np.random.seed(seed)
    np.random.shuffle(files)

    n = len(files)
    n_train = int(splits[0] * n)
    n_val   = int(splits[1] * n)
    train_files = files[:n_train]
    val_files   = files[n_train:n_train + n_val]
    test_files  = files[n_train + n_val:]

    for subset, flist in (('train', train_files),
                          ('val',   val_files),
                          ('test',  test_files)):
        subset_dir = os.path.join(output_dir, subset)
        os.makedirs(subset_dir, exist_ok=True)

        for fname in flist:
            src = os.path.join(input_dir, fname)
            dst = os.path.join(subset_dir, fname)

            if subset == 'train':
                process_file(src, dst,
                             target_gesture, target_len_per_run)
            else:
                shutil.copy2(src, dst)

        print(f"{subset}: {len(flist)} files → {subset_dir}")

if __name__ == "__main__":
    INPUT_DIR          = '/home/gerardo/LSE_SEGMENTATION_DATASET'
    OUTPUT_DIR         = '/home/gerardo/LSE_SEGMENTATION_SPLIT'
    TARGET_GESTURE     = 43
    TARGET_LEN_PER_RUN = 11

    split_and_prepare(INPUT_DIR, OUTPUT_DIR,
                      TARGET_GESTURE, TARGET_LEN_PER_RUN)


train: 3152 files → /home/gerardo/LSE_SEGMENTATION_SPLIT/train
val: 675 files → /home/gerardo/LSE_SEGMENTATION_SPLIT/val
test: 677 files → /home/gerardo/LSE_SEGMENTATION_SPLIT/test
